# Proyecto Modelado, Optimización y Simulación - Parte 2

## Autor: [Jaime Andres Torres Bermejo](https://github.com/mews6)

Seneca Libre se posiciona como líder en el comercio electrónico en Bogotá, comprometido
con una operación logística eficiente y competitiva. En la primera etapa de este proyecto,
desarrollamos un modelo matemático integral diseñado para optimizar las rutas de suministro,
enfrentando desaf ́ıos como alta demanda, costos elevados y las limitaciones de una flota
compuesta por m ́as de 700 vehıculos.  Este modelo se enfocó en reducir costos operativos,
mejorar la puntualidad de las entregas y equilibrar la carga de trabajo de los transportistas.
En esta segunda fase, el objetivo principal es implementar y validar el modelo dise ̃nado,
demostrando que refleja de manera precisa la operación logística y que es capaz de predecir
rutas y costos de forma confiable.  Cada equipo deberá implementar el modelo utilizando
herramientas  como
Python (Pyomo o PuLP ) o GAMS,  resolviendo  casos  de  prueba  que
simulan diferentes escenarios operacionales.  

A través de esta implementación, se espera
obtener resultados que permitan evaluar la viabilidad y precisión del modelo, as ́ı como
identificar oportunidades de mejora. La validación de esta etapa es clave, ya que los equipos analizarán no solo los resultados
de las rutas optimizadas, sino tambi ́en los costos involucrados, identificando discrepancias
entre el modelo y las operaciones reales.  Estos análisis permitir ́an a Seneca Libre tomar
decisiones estratégicas informadas, fortaleciendo su posicióen el mercado

In [ ]:
import os
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon
import geopandas as gpd
from concurrent.futures import ThreadPoolExecutor
import folium
from folium.plugins import MarkerCluster
from shapely.validation import explain_validity
from pyomo.environ import *
from pyomo.opt import SolverFactory


: 

# Data Preparation

In [ ]:
# Vehicles
drones = pd.read_csv('../data/case_1_base/drone_only.csv')
ev = pd.read_csv('../data/case_1_base/ev_only.csv')
gas_car = pd.read_csv('../data/case_1_base/gas_car_only.csv')

#Universals
loading_costs = pd.read_csv('../data/universal_loading_costs.csv')
vehicles_data = pd.read_csv('../data/universal_vehicles_data.csv')

#Clients-Depots
clients = pd.read_csv('../data/case_1_base/Clients.csv')
depots = pd.read_csv('../data/case_1_base/Depots.csv')



: 

### Vehicle data preparation

to make the 

In [19]:
vehicles = [drones, ev, gas_car]
vehicles = pd.concat(vehicles)
vehicles.replace('drone','Drone', inplace=True)
vehicles

,VehicleType,Capacity,Range
0,Drone,29.11,11.34
1,Drone,26.04,9.12
2,Drone,18.36,15.59
3,Drone,28.69,15.51
4,Drone,24.42,14.10
...,...,...,...
19,Gas Car,78.00,139.00
20,Gas Car,161.00,174.00
21,Gas Car,80.00,129.00
22,Gas Car,94.00,111.00


now we append relevant information into the data itself, and clean up a few of the results, for example, for us to be able to directly compare results, we will pass through a few conversions of units of measure, just to keep consistency.

In [ ]:
full_vehicles = vehicles.merge(vehicles_data, left_on='VehicleType', right_on='Vehicle')
full_vehicles = full_vehicles.fillna({1: 0})

#Convert Time Rate to daily
(full_vehicles["Time Rate [COP/min]"]*60*24)
full_vehicles.rename(columns={"Time Rate [COP/min]": "Time Rate [COP/day]"})

: 

In [ ]:
full_vehicles

: 

# Model Definition

## Restraints

$$
    min(\sum_{i}^{X} \sum_{j}^{X} y_{ijk} c_{ijk}) | \forall k \in X(V,N)|n \\
    \sum_{j}^{X} y_{bjk} = m ; b \in B | \forall k \in X(V,N)|n \\
    \sum_{i}^{X} y_{ibk} = m ; b \in B | \forall k \in X(V,N)|n \\
    u_i \in \mathbb{R} \quad \forall i \in X \\
    u_i - u_j + (n-1) \cdot x_{ijk} \leq n-2 \quad \forall i \neq j, \quad i,j \in X | \forall k \in X(V,N)|\\
    \sum_{i}^{X} \sum_{j}^{X} y_{ijk} = max(capacidad_{m}) | \forall k \in X(V,N)|n \\
    \sum_{i}^{X} \sum_{j}^{X} y_{ijk} = max(capacidad_{m}) | \forall k \in X(V,N)|n \\
        y_{iqk} - y_{qik} = 0 \forall i \in N, \forall k \in X(V,N)|n | q \in j \in N.\\
    \sum_{i}^{X} \sum_{j}^{X} \sum_{k}^{X} y_{ijk} = card(X)
$$

In [ ]:

from matplotlib import pyplot as plt


class RouteFindingModel:
    def __init__(self, drones, ev, gas_car, clients, depots, vehicles):
        """
        Initialize the model parameters.
        """

        self.drones = drones
        self.ev = ev
        self.gas_car = gas_car
        self.clients = clients
        self.depots = depots
        self.vehicles = vehicles
        
        self.install_costs = install_costs
        self.communication_costs = communication_costs
        self.adjacency_matrix = adjacency_matrix  # New adjacency matrix (C_z)

        # Create the Pyomo model
        self.model = ConcreteModel()

    def build_model(self):
        """
        Build the optimization model.
        """
        model = self.model

        # Sets
        model.sensor_types = Set(initialize=self.sensor_types)  # Different sensor types
        model.locations = Set(initialize=self.depots)  # Possible installation locations

        # Decision Variables
        model.x = Var(model.sensor_types, model.locations, domain=Binary)  # Binary variable for sensor placement
        model.y = Var(model.sensor_types, domain=NonNegativeIntegers)  # Total number of sensors of each t

        # Objective: Minimize total cost
        def obj_expression(model):
            energy_cost = sum(model.y[s] * self.energy_costs[s] for s in model.sensor_types)
            installation_cost = sum(model.x[s, l] * self.install_costs[l] for s in model.sensor_types for l in model.locations)
            communication_cost = sum(
                model.x[s, l] * self.communication_costs[l,s]
                for s in model.sensor_types for l in model.locations
            )
            return energy_cost + installation_cost + communication_cost

        model.obj = Objective(rule=obj_expression, sense=minimize)

        #Constraint: 

        # Constraints: Full coverage for each location with connection restriction
        model.full_coverage = ConstraintList()
        for l in model.locations:
            for s in model.sensor_types:
                if self.coverage_matrix[s, l] == 1:  # Sensor can cover location
                    # Ensure that each location is covered, but only if it is connected via the adjacency matrix
                    model.full_coverage.add(
                        sum(model.x[s, loc] for loc in model.locations if self.coverage_matrix[s, loc] == 1 and (loc, l) in self.adjacency_matrix and self.adjacency_matrix[loc, l] == 1) >= 1
                    )

        # Consistency Constraint: Total number of sensors of type s must match the placement decisions
        def sensor_consistency_rule(model, s):
            return model.y[s] == sum(model.x[s, l] for l in model.locations)

        model.sensor_consistency = Constraint(model.sensor_types, rule=sensor_consistency_rule)

        return model

    def solve_model(self):
        """
        Solve the model using the given solver.
        """
        solver = pyo.SolverFactory('highs' )
        results = solver.solve(self.model)
        return results

    def display_results(self):
        """
        Display the results of the optimization.
        """
        self.model.display()
    def print_output(self):
        """
        Plot sensor placement solutions on directed graphs using NetworkX for each sensor type in a subplot.
        """
        # Assign colors to sensor types
        sensor_type_colors = {
            self.sensor_types[0]: 'red',
            self.sensor_types[1]: 'green',
            self.sensor_types[2]: 'blue'
        }

        # Create subplots: 1 row, 3 columns (one for each sensor type)
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # Iterate over each sensor type
        for idx, s in enumerate(self.sensor_types):
            G = nx.DiGraph()  # Create a new directed graph for each sensor type

            # Add directed edges based on adjacency and sensor coverage for sensor type 's'
            for loc1 in self.locations:
                for loc2 in self.locations:
                    # Add a directed edge if the adjacency matrix indicates they are connected and covered by the sensor
                    if loc1 != loc2 and self.adjacency_matrix[loc1, loc2] == 1:
                        # Check if sensor type 's' is installed in loc1, and mark the edge color
                        if self.model.x[s, loc1]() == 1 and self.coverage_matrix[s, loc2] == 1:
                            G.add_edge(loc1, loc2, color=sensor_type_colors[s])  # Use sensor type color
                        else:
                            G.add_edge(loc1, loc2, color='black')  # Uncolored edges as black

            # Create node color mapping based on the decision variable
            node_colors = []
            for loc in G.nodes():
                if self.model.x[s, loc]() == 1:
                    node_colors.append(sensor_type_colors[s])  # Color nodes where this sensor is installed
                else:
                    node_colors.append('lightgray')  # Gray for nodes where this sensor is not installed

            # Get edge colors from the graph attributes
            edge_colors = [G[u][v]['color'] for u, v in G.edges]

            # Generate positions for nodes
            pos = nx.spring_layout(G)

            # Draw the directed graph with node and edge colors in the current axis
            nx.draw(G, pos, ax=axes[idx], node_color=node_colors, edge_color=edge_colors, with_labels=True, node_size=500, font_weight='bold', arrows=True)

            # Set the title for this subplot
            axes[idx].set_title(f"Sensor Type: {s}", fontsize=15)

        # Adjust layout to prevent overlapping
        plt.tight_layout()
        
        # Show the combined plot
        plt.show()